In [1]:
from selenium import webdriver
from xlwt import Workbook 
import pandas as pd
import time

In [2]:
def get_marks(s):
    num=""
    flag = 0
    for i in s:
        if(i=='\n'):
            flag = 1
        if flag==1:
            num = num + i
    return int(num)

In [3]:
def extract_data(data):
    wb = Workbook() 
    sheet1 = wb.add_sheet('IT') 
    sheet1.write(0, 0, 'ROLL NO') 
    sheet1.write(0, 1, 'NAME') 
    sheet1.write(0, 12, 'YGPA') 
    sheet1.write(0, 13, 'TOTAL') 
    sheet1.write(0, 14, 'PERCENTAGE') 
    
    driver = webdriver.Chrome('F:\Applications\chromedriver.exe')
    
    # driver.set_page_load_timeout(30)
    driver.set_window_size(1400,1000)
 
    driver.get('http://www.bietjhs.ac.in/result2019/GetResultodd.aspx')
    row = 1
    f=1
    
    for i in range(len(data)):
        rollno = data['Roll No'][i]
        name = data['Name'][i]
        col=0
        total =0
        col_row0 = 2
        
        time.sleep(1)
        # Roll no input 
        rln = driver.find_element_by_id('txtRollNo')
        rln.send_keys(int(rollno))
        
        
        # input semester
        option = driver.find_element_by_id('ddlSemester')
        for sem in option.find_elements_by_tag_name('option'):
            if(sem.get_attribute("value")=='3'):
                sem.click()
                
        # clicking 'submit' button
        submit = driver.find_element_by_id('btnSubmit')
        submit.click()
        
        
        # writting roll no and name in excel sheet
        sheet1.write(row, col, int(rollno))
        col += 1
            
            
        sheet1.write(row, col, name)
        col += 1
        
        
        # extracting ygpa
        ygpa=driver.find_element_by_id('lbloSGPA')
        
        
        # getting data from table
        table = driver.find_element_by_id('ctl04_ctl00_ctl00_grdViewSubjectMarksheet')
        tbody = table.find_element_by_tag_name('tbody')
        
        
        trc = 1
        for tr in tbody.find_elements_by_tag_name('tr'):
            trc += 1
            if(trc == 1):
                continue
            else:
                tdc=0
                for td in tr.find_elements_by_tag_name('td'):
                    tdc += 1
                    if(tdc==1):
                        spanc = 0
                        for span in td.find_elements_by_tag_name('span'):
                            spanc += 1
                            if(spanc == 2):
                                if(f == 1):
                                    # writting subject name ie. code in excel sheet 
                                    sheet1.write(0,col_row0,span.text)
                                    col_row0 += 1
                    if(tdc == 7):
                        marks = get_marks(td.text)
                        total += marks
                        sheet1.write(row, col, marks)
                        col += 1
        
        
        # writing 'ygpa' in excel sheet
        try: 
            sheet1.write(row, col, float(ygpa.text))
        except:
            sheet1.write(row, col, 0)
        col += 1
        
        # writing 'total' in excel sheet
        sheet1.write(row, col, total)
        col += 1
        
        
        # writing 'percentage' in excel sheet
        perc = float(total / 1025) * 100
        sheet1.write(row, col, "%.2f" % perc)
        col += 1
        
        print(name+"   "+ str("%.2f" % perc)+"%")
        
        driver.back()
        time.sleep(1)
        row += 1
        f = 0
        
    # save worksheet    
    wb.save('Marks.xlsx')
    # close driver
    driver.close()

In [4]:
data = pd.read_excel('IT-2nd.xlsx')
data

,Roll No,Name
0,1804313001,ABHILASH KUMAR
1,1804313002,ABHISHEK PAL
2,1804313003,ADARSH CHAUDHARY
3,1804313004,ADITE SINGH
4,1804313005,ADITI AGARWAL
5,1804313006,AKASH SHARMA
6,1804313007,AKSHAY PRATAP SINGH
7,1804313008,AMAN AGARWAL
8,1804313009,ARUNESH GAUTAM
9,1804313010,ASHOO PANDEY


In [5]:
extract_data(data)

ABHILASH KUMAR   72.78%
ABHISHEK PAL   71.80%
ADARSH CHAUDHARY   58.05%
ADITE SINGH   71.12%
ADITI AGARWAL   73.66%
AKASH SHARMA   59.22%
AKSHAY PRATAP SINGH   50.05%
AMAN AGARWAL   70.24%
ARUNESH GAUTAM   53.56%
ASHOO PANDEY   59.02%
AYUSHI CHOUDHARY   64.68%
BHAVUK SINGHAL   58.44%
DEVENDRA KUMAR BHARDWAJ   0.00%
DHEERENDRA GUPTA   61.07%
DUSHYANT KUMAR SINGH   53.37%
GAURI RAWAT   55.51%
GUNJAN BAGHEL   42.54%
HARSHITA ASHWANI   62.44%
MAYANK GIRI   66.54%
MONU KUMAR   55.90%
MUSKAN SINGHAL   67.02%
NAINCY PAHARIYA   49.95%
NEHARIKA SAXENA   64.29%
PRAVEG VASHISHTHA   76.10%
PREETI GUPTA   68.39%
PRIYANSHU PATHAK   63.51%
RUCHIKA MISHRA   69.66%
SAMBHAV JAIN   66.44%
SAURABH KUMAR SINGH   64.59%
SAURABH KUMAR SONKAR   36.20%
SHARAD KUMAR SINGH   72.59%
SHIVANI SHINGH   65.66%
SHUBHAM GUPTA   68.59%
SHUBHAM GUPTA   66.54%
SHUBHAM SHARMA   38.34%
SRIDHAR BAJPAI   61.46%
SUMI JAIN   0.00%
SUNIL KUMAR PATEL   55.90%
UMANG GUPTA   66.34%
VINEET KUMAR VERMA   54.05%
VIPUL SHARMA   48.78%


In [6]:
# sort by percentage

from pandas import ExcelWriter
import os
import numpy as np

df = pd.read_excel('Marks.xlsx')
df = df.replace(np.nan, 0)
df = df.sort_values('PERCENTAGE', ascending = False)


# Delete old sheet
os.remove("Marks.xlsx")

# save sheet
df.to_excel('Marks.xlsx', sheet_name  = 'IT',index=False, engine='xlsxwriter')  

# show dataframe
df


,ROLL NO,NAME,Technical Communication,Mathematics-IV,Data Structure,Computer Organization and Architecture,Discrete Structures and Theory of Logic,Data Structure Using C Lab,Computer Organization Lab,Discrete Structures and Logic Lab,MINI PROJECT OR INTERNSHIP ASSESSMENT,Computer System Security,YGPA,TOTAL,PERCENTAGE
23,1804313024,PRAVEG VASHISHTHA,128,107,129.0,103.0,96.0,42.0,41.0,40.0,41.0,53.0,8.18,780.0,76.10
4,1804313005,ADITI AGARWAL,103,100,118.0,98.0,120.0,43.0,40.0,40.0,42.0,51.0,7.82,755.0,73.66
0,1804313001,ABHILASH KUMAR,77,136,115.0,93.0,105.0,45.0,43.0,42.0,40.0,50.0,8.14,746.0,72.78
30,1804313031,SHARAD KUMAR SINGH,92,122,110.0,100.0,103.0,45.0,39.0,41.0,42.0,50.0,7.91,744.0,72.59
1,1804313002,ABHISHEK PAL,96,99,116.0,97.0,112.0,41.0,41.0,43.0,43.0,48.0,7.68,736.0,71.80
3,1804313004,ADITE SINGH,111,96,101.0,86.0,122.0,41.0,39.0,40.0,43.0,50.0,7.55,729.0,71.12
7,1804313008,AMAN AGARWAL,117,105,103.0,87.0,109.0,35.0,37.0,37.0,43.0,47.0,7.50,720.0,70.24
26,1804313027,RUCHIKA MISHRA,121,105,90.0,87.0,101.0,41.0,39.0,39.0,43.0,48.0,7.55,714.0,69.66
32,1804313033,SHUBHAM GUPTA,89,125,102.0,74.0,109.0,39.0,37.0,36.0,44.0,48.0,7.23,703.0,68.59
24,1804313025,PREETI GUPTA,92,92,90.0,96.0,100.0,44.0,45.0,46.0,43.0,53.0,7.45,701.0,68.39
